In [1]:
from PIL import Image # used for loading images
import numpy as np
import os # used for navigating to image path
import imageio # used for writing images
import random
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import SGD
import pydot
from timeit import default_timer as timer
from tensorflow.keras import backend as K

"ResNet 50 dependencies"
from tensorflow.keras.applications.resnet50 import ResNet50 
from tensorflow.keras.applications import resnet50

"GoogLeNet dependencies"
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator

"""HP Tuning"""
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

from tensorflow import keras

import IPython

from sklearn.preprocessing import LabelEncoder

import kerastuner as kt
from sklearn import ensemble
from sklearn import datasets
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection

In [2]:
"""Load numpy output files"""
pr_im_64 = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_Im.npy', allow_pickle=True)
pr_po_im_64 = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_PoIm.npy', allow_pickle=True)
pr_po_64 = np.load('../../data/tidy/preprocessed_images/size64_exp5_PrPo_Im.npy', allow_pickle=True)

In [2]:
def getImageOneHotVector(image_file_name, classification_scenario = "B"):
    """Returns one-hot vector encoding for each image based on specified classification scenario:
    Classification Scenario A (3 classes): {probable, possible, improbable}
    Classification Scenario B (2 classes): {probable, improbable}
    Classification Scenario C (2 classes): {{probable, possible}, improbable}
    Classification Scenario D (2 classes): {probable, {possible, improbable}}
    """
    word_label = image_file_name.split('-')[0]
    if classification_scenario == "A":
        if word_label == 'probable' : 
            return np.array([1, 0, 0])
        elif word_label == 'possible' : 
            return np.array([0, 1, 0])    
        elif word_label == 'improbable':
            return np.array([0, 0, 1])
        else :
            return np.array([0, 0, 0]) # if label is not present for current image
    elif classification_scenario == "B":
        if word_label == 'probable' : 
            return np.array([1, 0])
        elif word_label == 'improbable' : 
            return np.array([0, 1])
        else :
            return np.array([0, 0]) # if label is not present for current image
    elif classification_scenario == "C":
        if word_label in ['probable', 'possible'] : 
            return np.array([1, 0])
        elif word_label == 'improbable' : 
            return np.array([0, 1])
        else :
            return np.array([0, 0]) # if label is not present for current image        
    elif classification_scenario == "D":
        if word_label == 'probable' : 
            return np.array([1, 0])
        elif word_label in ['possible', 'improbable'] : 
            return np.array([0, 1])
        else :
            return np.array([0, 0]) # if label is not present for current image        

In [54]:
#IMG_SIZE = 300
NUM_CLASS = 2
NUM_CHANNEL = 1
CLASSIFICATION_SCENARIO = "B"

In [3]:
#IMG_SIZE = 300
NUM_CLASS = 2
NUM_CHANNEL = 1
CLASSIFICATION_SCENARIO = "B"
DIR = '../../data/tidy/labeled_images'
def processImageData(img_size, channels=1, l=400,t=0,r=3424,b=3024):    
    data = []
    image_list = os.listdir(DIR)
    for img in image_list:
        label = getImageOneHotVector(img, CLASSIFICATION_SCENARIO)
        if label.sum() == 0:
            continue
        path = os.path.join(DIR, img)
        img = Image.open(path)
        if channels == 1:
            img = img.convert('L') # convert image to monochrome 
            img = img.crop((l, t, r, b)) # after cropping, image size is 3024 x 3024 pixels
            #img_size_w, img_size_h = img.size
            img = img.resize((img_size, img_size), Image.BICUBIC)
            data.append([(np.array(img)/255.).T, label])#scale to 0-1 and transpose
#             flip_img = np.fliplr((np.array(img)/255.).T)# Basic Data Augmentation - Horizontal Flipping
#             data.append([flip_img, label])#scale to 0-1 and transpose
        elif channels == 3:
            img = img.crop((l, t, r, b)) # after cropping, image size is 3024 x 3024 pixels  
            img = img.resize((img_size, img_size), Image.BICUBIC)
            data.append([(np.array(img)/255.).T, label])#scale to 0-1 and transpose            
    return (data)

In [55]:
def splitData(image_array, prop = 0.80, seed_num = 111):
    """Returns training and test arrays of images with specified proportion - prop:1-prop"""
    random.Random(seed_num).shuffle(image_array)
    train_size = int(prop*np.shape(image_array)[0])
    train = image_array[:train_size]
    test = image_array[train_size:]
    return(train, test)

In [4]:
processed_image_data = processImageData(108, channels = NUM_CHANNEL)
train_data, test_data = splitData(processed_image_data, seed_num = 111) 

In [56]:
tr_data, te_data = splitData(pr_im_64, seed_num = 111)

In [5]:
processed_image_data[202][1]

In [75]:
plt.imshow(processed_image_data[202][0], cmap = 'gist_gray')
#plt.savefig( "../../figures/image0.png", dpi=100)

In [57]:
def getImageShape(image_array):
    if NUM_CHANNEL==1:
        image_shape = np.array([np.expand_dims(x[0],axis=2) for x in image_array]).shape[1:4]
    elif NUM_CHANNEL==3:
        image_shape = np.array([x[0] for x in image_array]).shape[1:4][::-1]
    print(image_shape)
    return image_shape

In [58]:
input_image_shape = getImageShape(tr_data)

In [59]:
# Train the model.
if NUM_CHANNEL == 1:
    train_array = np.array([np.expand_dims(x[0],axis=2) for x in tr_data])
    validation_array = np.array([np.expand_dims(x[0],axis=2) for x in te_data])
elif NUM_CHANNEL == 3:
    train_array = np.array([x[0] for x in tr_data]) 
    train_array = np.moveaxis(train_array, 1, -1)
    validation_array = np.array([x[0] for x in te_data])
    validation_array = np.moveaxis(validation_array, 1, -1)

train_labels = np.array([x[1] for x in tr_data])
validation_labels = np.array([x[1] for x in te_data])

In [22]:
#https://github.com/keras-team/keras/issues/5400#issuecomment-408743570
def check_units(y_true, y_pred):
    if y_pred.shape[1] != 1:
      y_pred = y_pred[:,1:2]
      y_true = y_true[:,1:2]
    return y_true, y_pred

def precision(y_true, y_pred):
    y_true, y_pred = check_units(y_true, y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    y_true, y_pred = check_units(y_true, y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    y_true, y_pred = check_units(y_true, y_pred)
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [11]:
def plot_model_accuracy(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("Model Accuracy")
    plt.ylabel("Accuracy")
    plt.xlabel("Epoch")
    plt.legend(["Train", "Validation"], loc="lower right")
    plt.show()

def plot_model_loss(hist):
    plt.plot(hist.history["loss"])
    plt.plot(hist.history["val_loss"])
    plt.title("Model Loss")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend(["Train", "Validation"], loc="upper right")
    plt.show()

In [12]:
# opt = keras.optimizers.Adam(learning_rate=0.01)
# model.compile(loss='categorical_crossentropy', optimizer=opt)

model = models.Sequential([
    layers.Conv2D(filters = 64, kernel_size = 7, strides = 2, activation="relu", padding="same", input_shape = input_image_shape),
    layers.MaxPooling2D(2),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5), # randomly drop out 50% of the neuorns at each training step
    layers.Dense(64, activation="relu"), # flatten all outputs
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASS, activation="softmax")
])

In [30]:
# Compile the model.
opt = SGD(lr = 0.001) #default learning rate (lr) = 0.1
model.compile(loss='categorical_crossentropy',  optimizer = "adam",
              metrics=[precision,recall, f1, 'accuracy'])

start = timer()
hist_seq = model.fit(
    train_array, train_labels, 
    batch_size = 32,
    epochs = 5,
    validation_data=(validation_array, validation_labels)
)
end = timer()
print(end - start) # Time in seconds, e.g. 5.380919524002

In [31]:
plot_model_accuracy(hist_seq)

In [33]:
plot_model_loss(hist_seq)

## HP Tuning

In [60]:
def model_builder(hp):
  model = models.Sequential()
  
  ## Vary kernel size in first Conv Layer between 5 and 7
  hp_k_size = hp.Choice('kernel_size', values = [5, 7])
  
  # Tune the number of units in the first and second Dense layers
  # Choose an optimal value between 32-512
  hp_units_l1 = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  hp_units_l2 = hp.Int('units', min_value = 32, max_value = 512, step = 32)  
    
  dropout_rate_a = hp.Float('dropout', min_value = 0.0, max_value = 0.5, step = 0.1)
  dropout_rate_b = hp.Float('dropout', min_value = 0.0, max_value = 0.5, step = 0.1)
    
  # Experiment with "relu" and "tanh" activation f-ns
  hp_dl1_activation = hp.Choice('activation', values = ['relu', 'tanh'])
  hp_dl2_activation = hp.Choice('activation', values = ['relu', 'tanh'])
    
  model.add(layers.Conv2D(filters = 64, kernel_size = hp_k_size, strides = 2, activation="relu", padding="same", input_shape = input_image_shape))

  model.add(layers.MaxPooling2D(2))
  model.add(layers.Conv2D(128, 3, activation="relu", padding="same"))
  model.add(layers.Conv2D(128, 3, activation="relu", padding="same"))
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Conv2D(256, 3, activation="relu", padding="same"))
  model.add(layers.Conv2D(256, 3, activation="relu", padding="same"))
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Flatten())
  
  
  model.add(layers.Dense(units = hp_units_l1, activation = hp_dl1_activation))
  model.add(layers.BatchNormalization()) # Networks train faster & converge much more quickly
  model.add(layers.Dropout(dropout_rate_a))
  model.add(layers.Dense(units = hp_units_l2, activation = hp_dl2_activation))
  model.add(layers.Dropout(dropout_rate_b))
  model.add(keras.layers.Dense(NUM_CLASS, activation='softmax'))
    
  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.CategoricalCrossentropy(from_logits = True), #keras.losses.SparseCategoricalCrossentropy(from_logits = True)
                metrics = ['accuracy'])
  return model

In [61]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_loss', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [62]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [63]:
tuner.search(train_array, train_labels, epochs = 10, validation_data = (validation_array, validation_labels), callbacks = [ClearTrainingOutput()])

In [51]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 10)[0]

#Best Hperparamter summary
best_hps_summary = f"""
The hyperparameter search for classification scenario {CLASSIFICATION_SCENARIO} is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')}. The optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}. The optimal kernel size of the first convolution layer is {best_hps.get('kernel_size')}.
The optimal dropout rate for the optimizer is {best_hps.get('dropout')}. The optimal activation layer for the optimizer is {best_hps.get('activation')}.
"""
print(best_hps_summary,  file=open('../../results/models/best_hyperparameters_scenario_b.txt', 'w'))

In [64]:
#Return best hyperparameter dictionary
best_hps_dict = tuner.get_best_hyperparameters()[0].values
best_hps_dict

In [66]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()
model.save('../../results/models/optimized_scenario_b')

In [23]:
model.compile(loss='binary_crossentropy', metrics=[precision,recall, f1, 'accuracy'])

In [32]:
start = timer()
model.fit(train_array, train_labels, epochs=10, validation_data=(validation_array, validation_labels))
end = timer()
print(end-start)

In [21]:
#best_model = tuner.get_best_models(num_models=1)[0]
model.evaluate(train_array, train_labels)

In [15]:
#loss, acc = model.evaluate(testImages, testLabels, verbose = 0)
#print(acc * 100)
y_pred = model.predict(validation_array, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

#print(classification_report(validation_labels, y_pred_bool))

In [59]:
tf.keras.utils.plot_model(model, "../../figures/cnn-model1.png", expand_nested = False, rankdir = "TB", show_shapes=True, dpi=192)

## ResNet 34

In [16]:
class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.main_layers = [
            tf.keras.layers.Conv2D(filters, 3, strides=strides, padding="same", use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(filters, 3, strides=1, padding="same", use_bias=False),
            tf.keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                tf.keras.layers.Conv2D(filters, 1, strides=strides, padding="same", use_bias=False),    
                tf.keras.layers.BatchNormalization()
            ]
    
    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [17]:
resnet34mod = tf.keras.models.Sequential()
resnet34mod.add(tf.keras.layers.Conv2D(64, 7, strides=2, input_shape=input_image_shape, padding="same", use_bias=False))
resnet34mod.add(tf.keras.layers.BatchNormalization())
resnet34mod.add(tf.keras.layers.Activation("relu"))
resnet34mod.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    resnet34mod.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
resnet34mod.add(tf.keras.layers.GlobalAvgPool2D())
resnet34mod.add(tf.keras.layers.Flatten())
resnet34mod.add(tf.keras.layers.Flatten())
resnet34mod.add(tf.keras.layers.Dense(NUM_CLASS, activation="softmax"))

In [18]:
resnet34mod.compile(loss='binary_crossentropy',  optimizer = "adam",# metrics=[ 'accuracy']) #tf.keras.metrics.SpecificityAtSensitivity(0.5), tf.keras.metrics.SensitivityAtSpecificity(0.5), 
              metrics=[precision,recall, f1, 'accuracy']) #metrics=['accuracy'])

In [19]:
start = timer()
resnet34mod.fit(
    train_array, train_labels, 
    batch_size = 32,
    epochs = 4,
    validation_data=(validation_array, validation_labels)
)
end = timer()
print(end - start) # Time in seconds, e.g. 5.380919524002

## ResNet 50

In [12]:
"Define ResNet 50 model instance (Keras built-in)"
rn50 = resnet50.ResNet50(include_top=True, 
                           weights=None, 
                           input_tensor=None, 
                           input_shape=input_image_shape, 
                           pooling= 'max', 
                           classes=2)

rn50.summary()

In [34]:
"Configure the model with losses and metrics"
rn50.compile(loss='categorical_crossentropy',  optimizer = "adam",
              metrics=[precision,recall, f1, 'accuracy']) 

start = timer()

"Fit ResNet 50 to data"
hist_rn50 = rn50.fit(
    train_array, train_labels, 
    batch_size = 32,
    epochs = 5,
    validation_data=(validation_array, validation_labels)
)

end = timer()
print(end - start)

In [35]:
plot_model_accuracy(hist_rn50)

In [36]:
plot_model_loss(hist_rn50)

## GoogLeNet
### InceptionV3

In [25]:
"""Instantiate the Inception v3 architecture"""
iv3 = tf.keras.applications.InceptionV3(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_image_shape,
    pooling='avg',
    classes=2,
    classifier_activation="softmax",
)

In [26]:
"Configure the model with losses and metrics"
iv3.compile(loss='categorical_crossentropy',  optimizer = "adam",
              metrics=[precision,recall, f1, 'accuracy']) 

start = timer()

"Fit Inception v3 to data"
hist_iv3 = iv3.fit(
    train_array, train_labels, 
    batch_size = 32,
    epochs = 10,
    validation_data=(validation_array, validation_labels)
)

end = timer()
print(end - start)

In [27]:
plot_model_accuracy(hist_iv3)

In [28]:
plot_model_loss(hist_iv3)